In [9]:
safety <- read.csv('https://raw.githubusercontent.com/IAA-Faculty/statistical_foundations/master/safety.csv')

In [10]:
library(tidyverse)

Build a logistic regression, predicting Unsafe using the variables Region, Weight, and Size. Treat Weight as a continuous variable. Treat Region and Size as categorical. Make sure to use the factor function for Size. 

a. Which variables were significant at the 0.05 level? 

Size

b. What is the concordance proportion for this model?

0.8477

In [11]:
safety  %>% str()
safety <- safety  %>% mutate(across(c(Unsafe, Size, Region, Type)))

'data.frame':	96 obs. of  5 variables:
 $ Unsafe: int  0 0 0 0 0 0 0 0 0 0 ...
 $ Size  : int  2 3 2 1 2 2 3 2 2 1 ...
 $ Weight: int  3 4 3 3 3 3 4 3 4 3 ...
 $ Region: chr  "N America" "N America" "N America" "N America" ...
 $ Type  : chr  "Medium" "Sport/Utility" "Medium" "Small" ...


In [12]:
unsafe_glm <- glm(Unsafe ~ Region + Weight + Size, data = safety)

In [13]:
unsafe_glm  %>% summary()


Call:
glm(formula = Unsafe ~ Region + Weight + Size, data = safety)

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)      1.03612    0.16753   6.185 1.69e-08 ***
RegionN America -0.03606    0.08763  -0.412    0.682    
Weight          -0.05675    0.06034  -0.940    0.349    
Size            -0.26194    0.06066  -4.318 3.96e-05 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for gaussian family taken to be 0.1569518)

    Null deviance: 20.625  on 95  degrees of freedom
Residual deviance: 14.440  on 92  degrees of freedom
AIC: 100.58

Number of Fisher Scoring iterations: 2


In [15]:
library(survival) 
survival::concordance(unsafe_glm)

Call:
concordance.lm(object = unsafe_glm)

n= 96 
Concordance= 0.8477 se= 0.03952
concordant discordant     tied.x     tied.y    tied.xy 
      1621        244        115       2273        307 

In [17]:
full.model <- glm(Unsafe ~ Region + Weight + Size, data = safety, family = binomial(link = "logit"))
empty.model <- glm(Unsafe ~ 1, data = safety, family = binomial(link = "logit"))

back.model <- step(full.model, 
                   scope = list(lower = formula(empty.model), 
                                upper = formula(full.model)), 
                                direction = "backward", 
                                k = qchisq(.05, 1, lower.tail = FALSE))

Start:  AIC=100.45
Unsafe ~ Region + Weight + Size

         Df Deviance     AIC
- Region  1   85.374  96.898
- Weight  1   86.989  98.514
<none>        85.085 100.451
- Size    1  101.329 112.854

Step:  AIC=96.9
Unsafe ~ Weight + Size

         Df Deviance     AIC
- Weight  1   87.343  95.026
<none>        85.374  96.898
- Size    1  102.764 110.447

Step:  AIC=95.03
Unsafe ~ Size

       Df Deviance     AIC
<none>      87.343  95.026
- Size  1  119.249 123.090


In [18]:
survival::concordance(back.model)

Call:
concordance.lm(object = back.model)

n= 96 
Concordance= 0.8182 se= 0.043
concordant discordant     tied.x     tied.y    tied.xy 
      1392        132        456       1539       1041 

In [1]:
100*(exp(cbind(coef(back.model), confint(back.model)))-1)

ERROR: Error in eval(expr, envir, enclos): object 'back.model' not found
